In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

In [2]:
center_locations = pd.read_csv("center_location.csv", index_col="Center")
landfill_locations = pd.read_csv("landfill_location.csv", index_col="Landfill")
center_info = pd.read_csv("center_info.csv", index_col="Center")
center_locations.head()
landfill_locations.head()
center_info.head()

,waste tonnage,average income,population
Center,,,
Sioux Falls,260,44430,6466
Apison,525,38929,15929
Grandin,571,32742,13866
Southern Gateway,408,49612,11569
Seattle,183,31373,4578


In [3]:
centers = list(center_locations.index)
landfills = list(landfill_locations.index)

In [4]:
sl_distance_centers_landfills = pd.DataFrame(index=centers, columns=landfills)
for i in centers:
    for j in landfills:
        center_x = center_locations.loc[i, "x"]
        center_y = center_locations.loc[i, "y"]
        landfill_x = landfill_locations.loc[j, "x"]
        landfill_y = landfill_locations.loc[j, "y"]
        distance = ((center_x - landfill_x)**2 +
                    (center_y - landfill_y)**2)**(1/2)
        sl_distance_centers_landfills.loc[i, j] = distance

In [5]:
# calculate big-M value
bigM = max(center_info["waste tonnage"])

# Create the model object (for question 1b)
model_question1b = gp.Model("question1b_model")

# Add the decision variables
xcl = model_question1b.addVars(centers, landfills, name="xcl")
z = model_question1b.addVars(landfills, vtype=GRB.BINARY, name="z")

# Construct the objective function 
objective_function = model_question1b.setObjective(
    1.03 * sum(sl_distance_centers_landfills.loc[i, j] * xcl[i, j] for i in centers for j in landfills), 
    GRB.MINIMIZE)

# Waste collection constraints:
# We must transport all waste away from each center
collect_waste_constraint = model_question1b.addConstrs(
    (sum(xcl[i, j] for j in landfills) == center_info.loc[i, 'waste tonnage'] for i in centers), 
    name="collect_waste")

# We can only transport waste to landfill locations that have been built
bibig_M_constraint = model_question1b.addConstrs(
    (xcl[i, j] <= bigM * z[j] for i in centers for j in landfills), 
    name="big_M")

# We can build at most 4 landfills
max_num_landfills_constraint = model_question1b.addConstr(
    sum(z[j] for j in landfills) <= 4, 
    name="max_num_landfills")

Restricted license - for non-production use only - expires 2025-11-24


In [6]:
model_question1b.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 22.6.0 22G74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 801 rows, 765 columns and 2265 nonzeros
Model fingerprint: 0xda3796a2
Variable types: 750 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 1e+03]
Presolve time: 0.01s
Presolved: 801 rows, 765 columns, 2265 nonzeros
Variable types: 750 continuous, 15 integer (15 binary)
Found heuristic solution: objective 1071357.2543

Root relaxation: objective 4.407073e+05, 54 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    440707.33563 440707.336  0.00%     -    0s

Explored 1 nodes 

In [7]:
built_landfills = []
for j in landfills:
    if z[j].x >= 0.9999:
        print("Build landfill at", j)
        built_landfills.append(j)
        
total_cost = 1.03 * sum(sl_distance_centers_landfills.loc[i,j]*xcl[i,j].x for i in centers for j in landfills)
print("Total weekly cost is: $ %f" % total_cost)

Build landfill at Lake Lorraine
Build landfill at Muhlenberg Park
Build landfill at Weiser
Build landfill at Renfrow
Total weekly cost is: $ 440707.335626
